In [14]:
pip install xgboost

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install bayesian-optimization

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install lightgbm

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install scikit-optimize==0.8.1

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [18]:
pip install scikit-learn==0.23.2

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [19]:
pip install catboost

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [20]:
import pickle
from sklearn.preprocessing import StandardScaler,MinMaxScaler
import matplotlib.pyplot as plt

import seaborn as sns

import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import  LogisticRegression
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score,GridSearchCV
from skopt import BayesSearchCV

from sklearn.metrics import roc_curve, auc

from sklearn.model_selection import train_test_split

%matplotlib inline
plt.style.use('fivethirtyeight')

## Part 2: Supervised Learning Model


## Load and clean the train and test dataset

In [21]:

# I have renamed Udacity_MAILOUT_052018_TRAIN.csv and Udacity_MAILOUT_052018_TEST.csv for more readability
# load data from csv files to make operation on them
# missing values found in section of missing values and set it here for furure use
missing_values = ["X", "XX"]

#read csv file of mailout train with changing missing values to nan
mailout_train = pd.read_csv('../data/mail_train.csv',sep=';',na_values=missing_values)
#read csv file of mailout test with changing missing values to nan
mailout_test = pd.read_csv('../data/mail_test.csv',sep=';',na_values=missing_values)


In [22]:
#check if mailout train has more than 80% missing values

#sum of mailout_train null value
mailout_train_null_sum=mailout_train.isnull().sum()

#check percent of mailout train missing value
missing_columns=mailout_train_null_sum/len(mailout_train)

#list which features have missing value more than 80 percent
missing_percent=0.80
mailout_train[missing_columns[missing_columns > missing_percent].index]


,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
...,...,...,...,...
42957,NaN,NaN,NaN,NaN
42958,NaN,NaN,NaN,NaN
42959,NaN,NaN,NaN,NaN
42960,NaN,NaN,NaN,NaN


In [23]:
#check if mailout test has more than 80% missing values

#sum of mailout_test null value
mailout_test_null_sum=mailout_test.isnull().sum()

#check percent of mailout test missing value
missing_columns=mailout_test_null_sum/len(mailout_test)

#list which features have missing value more than 80 percent
missing_percent=0.80
mailout_test[missing_columns[missing_columns > missing_percent].index]


,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
...,...,...,...,...
42828,NaN,NaN,NaN,NaN
42829,NaN,NaN,NaN,NaN
42830,14.0,17.0,NaN,NaN
42831,NaN,NaN,NaN,NaN


In [24]:
mailout_train['ALTER_KIND1']=mailout_train['ALTER_KIND1'].fillna(mailout_train['ALTER_KIND1'].mean())

In [25]:
mailout_train['ALTER_KIND2']=mailout_train['ALTER_KIND2'].fillna(mailout_train['ALTER_KIND2'].mean())

In [26]:
mailout_train['ALTER_KIND3']=mailout_train['ALTER_KIND3'].fillna(mailout_train['ALTER_KIND3'].mean())

In [27]:
mailout_train['ALTER_KIND4']=mailout_train['ALTER_KIND1'].fillna(mailout_train['ALTER_KIND4'].mean())

In [28]:
mailout_test['ALTER_KIND1']=mailout_test['ALTER_KIND1'].fillna(mailout_test['ALTER_KIND1'].mean())

In [29]:
mailout_test['ALTER_KIND2']=mailout_test['ALTER_KIND2'].fillna(mailout_test['ALTER_KIND2'].mean())

In [30]:
mailout_test['ALTER_KIND3']=mailout_test['ALTER_KIND3'].fillna(mailout_test['ALTER_KIND3'].mean())

In [31]:
mailout_test['ALTER_KIND4']=mailout_test['ALTER_KIND1'].fillna(mailout_test['ALTER_KIND4'].mean())

In [32]:
#I have renamed DIAS Attributes - Values 2017.xlsx to attributes
# use openpyxl as an engine for reading xlsx files
# as far as first row of attributes file is empty I set skip first row of attributes
attributes = pd.read_excel('../data/attributes.xlsx', engine='openpyxl', skiprows = 1)


In [33]:
#first I have to list all values that recognize as unknow
unkown_list= attributes['Meaning'].where(attributes['Meaning'].str.contains('unknown')).value_counts().index

In [34]:
# make a list that have unknown values of each attribute
unkown_attributes_list = attributes[attributes['Meaning'].isin(unkown_list)]

In [35]:
def unknown_to_nan(dataframe):
   

    # rotate through all of unkown attributes
    for row in unkown_attributes_list.iterrows():
        
        # list the missed Value attributes from attributs excel file
        missed_values_list = row[1]['Value']
        
        # list the attributes from attributs excel file
        attribute = row[1]['Attribute']

        # to pass if columns is not in dataframe
        if attribute not in dataframe.columns:
            
            continue

        # to pass if value is integer
        if isinstance(missed_values_list, int):
            
            dataframe[attribute].replace(missed_values_list, -1, inplace=True)
         # to pass if value is String    
        elif isinstance(missed_values_list, str):
            
            #parses the expression to make it as integer -1 with unkown
            eval("dataframe[attribute].replace([" + missed_values_list + "], -1, inplace=True)")
            
            dataframe[attribute].replace(missed_values_list, -1, inplace=True)

    return dataframe

In [36]:
#change all mailout_train unkown value to fill with -1
mailout_train = unknown_to_nan(mailout_train)


In [37]:
#change all mailout_train unkown value to fill with -1
mailout_test = unknown_to_nan(mailout_test)


In [38]:
#cast datatype to float
def values_type_to_float(dataframe,column):
    
    dataframe[column] = dataframe[column].astype('float')

    
    return dataframe

In [39]:
#map the value to numerical value  
def values_map(dataframe,feature_old,feature_new,dictionary):
    
    dataframe[feature_new] = dataframe.loc[:, feature_old].map(dictionary)
    
    return dataframe

In [40]:
# make feature to have just year to mange it as numerical value
def year_extractor(dataframe,column):
    
    dataframe['EINGEFUEGT_AM'] = pd.to_datetime(dataframe['EINGEFUEGT_AM']).dt.year
    
    return dataframe


In [41]:
def drop_features(dataframe,features):

    dataframe.drop(features, axis=1,inplace=True)
    
    return dataframe

In [42]:
#step by step cleaning and transforming process
def clean_transform(dataframe,dataframe_name):
    
    #dataframe= values_replace(dataframe)
    print(dataframe_name,'replace object values : done')
    
    dataframe= values_type_to_float(dataframe,'CAMEO_DEUG_2015')
    print(dataframe_name,'cast values to float : done')
    
    dataframe= values_type_to_float(dataframe,'CAMEO_INTL_2015')
    print(dataframe_name,'cast values to float : done')
    

    dataframe= values_map(dataframe,'OST_WEST_KZ','OST_WEST_KZ_E',{'W': 0, 'O': 1})
    print(dataframe_name,'OST_WEST_KZ mapping : done')

    
    dataframe= year_extractor(dataframe,'EINGEFUEGT_AM')
    print(dataframe_name,'EINGEFUEGT_AM year extraction : done')

    
    dataframe= drop_features(dataframe,['LNR', 'D19_LETZTER_KAUF_BRANCHE', 'OST_WEST_KZ', 'CAMEO_DEU_2015'])
    print(dataframe_name,'drop some features : done')

    
    dataframe= values_map(dataframe,'PRAEGENDE_JUGENDJAHRE','PRAEGENDE_JUGENDJAHRE_DECADE',{1: 1, 2: 1, 3: 2, 4: 2, 5: 3, 6: 3, 7: 3, 8: 4, 9: 4, 10: 5,11: 5, 12: 5, 13: 5, 14: 6,15: 6, 0: np.nan})
    print(dataframe_name,'PRAEGENDE_JUGENDJAHRE mapping : done')

    
    dataframe= values_map(dataframe,'PRAEGENDE_JUGENDJAHRE','PRAEGENDE_JUGENDJAHRE_MOVEMENT',{1: 0, 2: 1, 3: 0, 4: 1, 5: 0, 6: 1, 7: 1, 8: 0, 9: 1, 10: 0, 11: 1, 12: 0, 13: 1, 14: 0, 15: 1, 0: np.nan})
    print(dataframe_name,'PRAEGENDE_JUGENDJAHRE mapping : done')

    
    dataframe= drop_features(dataframe,['PRAEGENDE_JUGENDJAHRE'])
    print(dataframe_name,'drop PRAEGENDE_JUGENDJAHRE as an old feature : done')


    print(dataframe_name,'cleaning and transforming finished')
    return dataframe

In [43]:
#start the proccess of clean mailout_train dataframe from object datatype and transform some features to new feature
mailout_train = clean_transform(mailout_train,'mailout_train')

mailout_train replace object values : done
mailout_train cast values to float : done
mailout_train cast values to float : done
mailout_train OST_WEST_KZ mapping : done
mailout_train EINGEFUEGT_AM year extraction : done
mailout_train drop some features : done
mailout_train PRAEGENDE_JUGENDJAHRE mapping : done
mailout_train PRAEGENDE_JUGENDJAHRE mapping : done
mailout_train drop PRAEGENDE_JUGENDJAHRE as an old feature : done
mailout_train cleaning and transforming finished


In [44]:
#start the proccess of clean mailout_test dataframe from object datatype and transform some features to new feature
mailout_test = clean_transform(mailout_test,'mailout_test')

mailout_test replace object values : done
mailout_test cast values to float : done
mailout_test cast values to float : done
mailout_test OST_WEST_KZ mapping : done
mailout_test EINGEFUEGT_AM year extraction : done
mailout_test drop some features : done
mailout_test PRAEGENDE_JUGENDJAHRE mapping : done
mailout_test PRAEGENDE_JUGENDJAHRE mapping : done
mailout_test drop PRAEGENDE_JUGENDJAHRE as an old feature : done
mailout_test cleaning and transforming finished


In [45]:
#here we create our train and test dataframe and dataframe of actual output for test it with our model

X_train = mailout_train

X_test = mailout_test

y_train = X_train['RESPONSE']


X_train.drop(['RESPONSE'], axis = 1, inplace = True)


# test models with different classifier and score with Roc_Auc

In [46]:
def train_model_compute_roc(classifier_model,classifier_name):
    
    #create pipline for models
    pipeline = Pipeline([('impute', SimpleImputer(strategy= 'constant', fill_value = -1)),('scale', StandardScaler()),  (classifier_name, classifier_model)])
    
    #split X_train and y_train to create random train and test set
    X_train_n, X_test_n, y_train_n, y_test_n = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)
    
    #fit model to make prediction in next step
    pipeline.fit(X_train_n, y_train_n)
    
    
    #tpr as rue positive rate and fpr as False positive rate or sensitivity
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    
    # make a prediction of y_score with X_test_n
    y_score_pred = pipeline.predict_proba(X_test_n)[:, 1]
    
    
    
    
    for n in range(2):
        
        fpr[n], tpr[n], _ = roc_curve(y_test_n, y_score_pred)
        
        roc_auc[n] = auc(fpr[n], tpr[n])
    
    
    return roc_auc
    

In [47]:
scores = []

In [48]:
scores.append(['XGBClassifier',train_model_compute_roc(XGBClassifier(),'XGBClassifier')[0]])


/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:23:04] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [49]:
scores.append(['LogisticRegression',train_model_compute_roc(LogisticRegression(),'Logistic Regression')[0]])


/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [50]:
scores.append(['RandomForestClassifier',train_model_compute_roc(RandomForestClassifier(),'RandomForestClassifier')[0]])


In [51]:
scores.append(['LGBMClassifier',train_model_compute_roc(LGBMClassifier(),'LGBMClassifier')[0]])


In [52]:
scores.append(['catboost',train_model_compute_roc(CatBoostClassifier(),'CatBoostClassifier')[0]])


Learning rate set to 0.046653
0:	learn: 0.5985155	total: 63.9ms	remaining: 1m 3s
1:	learn: 0.5180809	total: 77ms	remaining: 38.4s
2:	learn: 0.4499156	total: 91.7ms	remaining: 30.5s
3:	learn: 0.3923226	total: 106ms	remaining: 26.3s
4:	learn: 0.3435625	total: 122ms	remaining: 24.3s
5:	learn: 0.3023043	total: 138ms	remaining: 22.9s
6:	learn: 0.2677020	total: 152ms	remaining: 21.6s
7:	learn: 0.2386010	total: 167ms	remaining: 20.7s
8:	learn: 0.2139100	total: 183ms	remaining: 20.2s
9:	learn: 0.1932262	total: 196ms	remaining: 19.4s
10:	learn: 0.1755524	total: 210ms	remaining: 18.9s
11:	learn: 0.1606978	total: 225ms	remaining: 18.5s
12:	learn: 0.1478688	total: 240ms	remaining: 18.2s
13:	learn: 0.1369494	total: 255ms	remaining: 17.9s
14:	learn: 0.1276947	total: 271ms	remaining: 17.8s
15:	learn: 0.1197379	total: 287ms	remaining: 17.7s
16:	learn: 0.1129477	total: 303ms	remaining: 17.5s
17:	learn: 0.1071335	total: 315ms	remaining: 17.2s
18:	learn: 0.1020487	total: 329ms	remaining: 17s
19:	learn: 0

In [53]:
scores_dataframe = pd.DataFrame(scores, columns = ['Name', 'Score'])

In [54]:
scores_dataframe

,Name,Score
0,XGBClassifier,0.708335
1,LogisticRegression,0.700094
2,RandomForestClassifier,0.638744
3,LGBMClassifier,0.730610
4,catboost,0.757077


## bayesian XGBClassifier tunning 

In [55]:

#define limit of bayesian hyper parameter
bayesian_tunning_bounds = {
    'learning_rate': (0.00, 1.0),
    'n_estimators': (100, 110),
    'max_depth': (3,10),
    'gamma': (0, 1),
    'subsample':(0.5,0.9),
    'colsample':(0.5,0.9)}

In [56]:
cv=3
scoring_metric="roc_auc"

In [57]:
def XGBClassifier_optimization_function(learning_rate,
                        n_estimators,
                        max_depth,
                        gamma,
                       subsample,
                       colsample):

    max_depth = int(max_depth)
    n_estimators = int(n_estimators)
    

    classifier = XGBClassifier(
        max_depth=max_depth,
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        gamma=gamma,
        subsample=subsample,
        colsample=colsample,
        eta=0.1,
        eval_metric='auc')
    return np.mean(cross_val_score(classifier, X_train, y_train, cv=cv, scoring=scoring_metric))


In [58]:
XGBoost_opt = BayesianOptimization(
    f=XGBClassifier_optimization_function,
    pbounds=bayesian_tunning_bounds,
    random_state=1
   
)

In [59]:
%%time
#performing Bayesian optimization for 5 iterations with 10 steps of random exploration with an acquisition function of expected improvement
XGBoost_opt.maximize(n_iter=5, init_points=10, acq='ei')

|   iter    |  target   | colsample |   gamma   | learni... | max_depth | n_esti... | subsample |
-------------------------------------------------------------------------------------------------


/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:23:43] WARNING: ../src/learner.cc:573: 
Parameters: { "colsample" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:23:47] WARNING: ../src/learner.cc:573: 
Parameters: { "colsample" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




...

[20:31:52] WARNING: ../src/learner.cc:573: 
Parameters: { "colsample" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:32:06] WARNING: ../src/learner.cc:573: 
Parameters: { "colsample" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:32:20] WARNING: ../src/learner.cc:573: 
Parameters: { "colsample" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


|  15       |  0.602    |  0.8667   |  0.09461  |  0.9866   |  9.629    |  100.1    |  0.6816   |
CPU times: user 1h 9min 4s, sys: 7.02 s, total: 1h 9min 11s
Wall time: 8min 51s


In [61]:
#Extracting the best parameters for usage in kaggle model creation
params=XGBoost_opt.max['params']
print(params)

{'colsample': 0.8505556609184153, 'gamma': 0.8946066635038473, 'learning_rate': 0.08504421136977791, 'max_depth': 3.2733834826301766, 'n_estimators': 101.69830419564569, 'subsample': 0.8512570013717653}


In [62]:
#Converting the max_depth and n_estimator values from float to int
params['max_depth']= int(params['max_depth'])
params['n_estimators']=int(params['n_estimators'])

## Part 3: Kaggle 

In [64]:
number_of_features = 366

#Calculate Feature Importance
model_features_importance = XGBClassifier( verbose = False).fit( X_train, y_train)
features_importance = pd.DataFrame({'features':X_train.columns, 'importance' : model_features_importance.feature_importances_/ model_features_importance.feature_importances_.sum()}).sort_values(['importance'], ascending = False)


[20:34:43] WARNING: ../src/learner.cc:573: 
Parameters: { "verbose" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[20:34:43] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [66]:
#reduce the ftrain and test data based on the important features
X_train_kahesh = X_train[features_importance['features'].head(number_of_features).to_list()]
X_test_kahesh = X_test[features_importance['features'].head(number_of_features).to_list()]

In [67]:
mailout_test_LNR= pd.read_csv('../data/mail_test.csv',sep=';',na_values=missing_values)
mailout_test_LNR=unknown_to_nan(mailout_test_LNR)

In [69]:
#use the dictionaty with optimized parameters result of the BayesianOptimization
kaggle_model = XGBClassifier(**params)



#fit kaggle model with reduced x and y
kaggle_model.fit(X_train_kahesh, y_train)

y_sub = kaggle_model.predict_proba(X_test_kahesh)[:, 1]




[20:36:18] WARNING: ../src/learner.cc:573: 
Parameters: { "colsample" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[20:36:18] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [70]:
#create submit file for kaggle
submission = pd.DataFrame({'LNR': mailout_test_LNR['LNR'], 'RESPONSE': y_sub})
#save dataframe to csv file for submit
submission.to_csv('submission_xgboost.csv', index=False)